In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode, split, from_json, col, format_number
from pyspark.sql.types import StringType, FloatType, StructType, StructField

spark = SparkSession \
    .builder \
    .appName('LocalStreamSample') \
    .getOrCreate()

23/05/20 06:40:23 WARN Utils: Your hostname, mia-VirtualBox resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
23/05/20 06:40:23 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/05/20 06:40:24 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
# Create DataFrame for input Stream
data = spark \
    .readStream \
    .format('socket') \
    .option('host', 'localhost') \
    .option('port', '9999') \
    .load()

schema = StructType(
    [
        StructField('account_id', StringType(), True),
        StructField('account_region', StringType(), True),
        StructField('transaction_value', FloatType(), True),
    ]
)

# Stream Listening View
json_data = data.withColumn('json_data', from_json(col('value'), schema)) \
                .select('json_data.*')

# Credit Flow by Region View
# Comment for Stream Listening View
region_data = json_data.groupBy('account_region').sum() \
                .withColumn('credit_flow', format_number('sum(transaction_value)', 2)) \
                .select(['account_region', 'credit_flow'])

23/05/20 06:40:25 WARN TextSocketSourceProvider: The socket source should not be used for production applications! It does not support recovery.


In [3]:
# query = json_data \
query = region_data \
    .writeStream \
    .outputMode("complete") \ # outputMode("append") for Stream Listening View
    .format('console') \
    .start()

query.awaitTermination()

23/05/20 06:40:28 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-de604aa0-1037-42b0-8a86-82a3d8d668c7. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
23/05/20 06:40:28 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+--------------+-----------+
|account_region|credit_flow|
+--------------+-----------+
+--------------+-----------+



-------------------------------------------
Batch: 1
-------------------------------------------
+-----------------+-----------+
|   account_region|credit_flow|
+-----------------+-----------+
|    East Asian TZ|   5,452.89|
|North American TZ|  15,087.78|
+-----------------+-----------+



-------------------------------------------
Batch: 2
-------------------------------------------
+-----------------+-----------+
|   account_region|credit_flow|
+-----------------+-----------+
|    East Asian TZ|   5,945.32|
|North American TZ|  20,884.54|
+-----------------+-----------+



-------------------------------------------
Batch: 3
-------------------------------------------
+-----------------+-----------+
|   account_region|credit_flow|
+-----------------+-----------+
|    East Asian TZ|   5,945.32|
|North American TZ|  46,314.33|
+-----------------+-----------+



...

-------------------------------------------
Batch: 240
-------------------------------------------
+-----------------+-----------+
|   account_region|credit_flow|
+-----------------+-----------+
|  African Free TZ| 233,313.98|
|    East Asian TZ| 417,703.14|
|       Eurasia TZ| 288,848.81|
|North American TZ| 656,392.27|
|   South Asian TZ| 103,041.24|
+-----------------+-----------+



-------------------------------------------
Batch: 241
-------------------------------------------
+-----------------+-----------+
|   account_region|credit_flow|
+-----------------+-----------+
|  African Free TZ| 233,313.98|
|    East Asian TZ| 420,404.38|
|       Eurasia TZ| 288,848.81|
|North American TZ| 656,392.27|
|   South Asian TZ| 110,422.77|
+-----------------+-----------+



ERROR:root:KeyboardInterrupt while sending command.=========>   (186 + 4) / 200]
Traceback (most recent call last):
  File "/home/mia/.local/lib/python3.10/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/mia/.local/lib/python3.10/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt
ERROR:root:KeyboardInterrupt while sending command.=========>   (186 + 4) / 200]
Traceback (most recent call last):
  File "/home/mia/.local/lib/python3.10/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/mia/.local/lib/python3.10/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.10/socke

-------------------------------------------
Batch: 242
-------------------------------------------
+-----------------+-----------+
|   account_region|credit_flow|
+-----------------+-----------+
|  African Free TZ| 239,745.41|
|    East Asian TZ| 420,404.38|
|       Eurasia TZ| 288,848.81|
|North American TZ| 656,392.27|
|   South Asian TZ| 110,422.77|
+-----------------+-----------+



KeyboardInterrupt: 